# Evn

In [ ]:
# imports
import argparse
import os
import random
import shutil
import json
import zipfile
import math
import copy
import collections
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sentencepiece as spm
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm.notebook import tqdm, trange

In [ ]:
# 환경 설정
args = {
    # random seed value
    "seed": 1234
}
args = argparse.Namespace(**args)

print(args)

In [ ]:
# random seed 설정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
# gpu 사용량 확인
!nvidia-smi

In [ ]:
# data dir (환경에 맞게 수정하세요.)
data_dir = '/content/drive/MyDrive/문서/강의계획서/삼성전기/삼성전기.20220228/data'
os.listdir(data_dir)

# Vocabulary*

In [ ]:
# vocab loading
vocab = spm.SentencePieceProcessor()
vocab.load(os.path.join(data_dir, 'kowiki', 'kowiki_32000.model'))

# One-hot

In [ ]:
# one hot encoding text
text = """나는 학생 입니다
나는 좋은 선생님 입니다
당신은 매우 좋은 선생님 입니다"""

In [ ]:
# 줄바꿈 단위로 문장 분리
sentences = text.split("\n")
sentences

In [ ]:
# 띄어쓰기 단위로 단어 분리
tokens = []
for sentence in sentences:
    tokens.append(vocab.encode_as_pieces(sentence))
tokens

In [ ]:
# tokens을 vocabulary의 고유 번호로 변경
token_ids = []
for line_token in tokens:
    token_ids.append(vocab.piece_to_id(line_token))
token_ids

In [ ]:
# one hot encoding
one_hot_encodings = []
for line_token in token_ids:
    print(line_token)
    one_hot_line = []  # 한 줄을 표현하는 벡터
    for id in line_token:
        one_hot = [0] * len(vocab)  # 모두 0인 벡터를 만듬
        one_hot[id] = 1  # 단어 id만 1로 변경
        print(id, one_hot)
        one_hot_line.append(one_hot)  # 단어를 라인에 추가
    one_hot_encodings.append((one_hot_line))  # 라인을 전체 문서에 추가

In [ ]:
np.argmax(np.array(one_hot_encodings[1]), axis=-1)

In [ ]:
# tensorflow one hot
# token_ids가 길이가 달라서 오류 발생
# depth는 vocabulary 크기
tf_one_hot_encodings = tf.one_hot(indices=token_ids, depth=len(vocab))
tf_one_hot_encodings

In [ ]:
# token_ids가 앞 1개만 one_hot으로 변경
tf_one_hot_encodings = tf.one_hot(indices=token_ids[:1], depth=len(vocab))
tf_one_hot_encodings

In [ ]:
# 모두 길이가 5가 되도록 pad(0) 추가
pad_ids = []
for line in token_ids:
    line = line[:5]
    line += [0] * (5 - len(line))
    pad_ids.append(line)
pad_ids

In [ ]:
tf_one_hot_encodings = tf.one_hot(indices=pad_ids, depth=len(vocab))
tf_one_hot_encodings

# Embedding

In [ ]:
# 랜덤 매트릭스 생성
weights = np.random.randint(10, 100, size=(len(vocab), 4)) / 100
weights

In [ ]:
# 첫번째 문장 만 numpy array로 변경
one_hot_encoding_0 = np.array(one_hot_encodings[0])
one_hot_encoding_0

In [ ]:
# one hot encoding은 matrix의 특정 row를 선택하는 것과 같은 결과
hidden = np.matmul(one_hot_encoding_0, weights)
hidden

In [ ]:
print(token_ids[0])
print(weights[token_ids[0][0]])
print(weights[token_ids[0][1]])
print(weights[token_ids[0][2]])
print(weights[token_ids[0][3]])

In [ ]:
# tensorflow 에서도 tf.keras.layers.Embedding에서도 가능 함
# weights 초기화를 위해서 [matrix] 형태로 변환함
# embdding을 사용하면 token 번호를 바로 사용가능 함 현재 표준화된 방법
embedding = tf.keras.layers.Embedding(len(vocab), 4, weights=[weights])

In [ ]:
embedding(np.array(token_ids[0]))

In [ ]:
tf.gather(weights, token_ids[0])

# 실습
- """나는 국어를 좋아합니다  
나는 수학을 싫어합니다  
나는 과학을 매우 많이 좋아합니다""" 위 문장을 이용해 one-hot으로 변경해 보세요.
- np.random.randn 함수를 이용해 embedding matrix를 선언하고 만들어진 one-hot과 행렬곱을 이용해 벡터로 변경해 보세요.
- tf의 embedding을 이용해서 단어를 벡터로 변경해 보세요.
- tf의 gather를 이용해 단어를 벡터로 변경해 보세요.




# Cross Entropy Loss

In [ ]:
y_true = np.random.randint(0, 2, (5,))
y_true

In [ ]:
y_pred = np.random.randn(5, 2)
y_pred

In [ ]:
y_pred = tf.nn.softmax(y_pred, axis=-1)
y_pred

In [ ]:
y_true_p = tf.one_hot(y_true, 2)
y_true_p

In [ ]:
n_log_p = -np.log(y_pred)
n_log_p

In [ ]:
ce1 = y_true_p * n_log_p
ce1

In [ ]:
ce2 = np.sum(ce1, axis=-1)
ce2

In [ ]:
loss1 = np.mean(ce2)
loss1

In [ ]:
loss2 = tf.keras.losses.CategoricalCrossentropy()(y_true_p, y_pred)
loss2

In [ ]:
loss3 = tf.keras.losses.SparseCategoricalCrossentropy()(y_true, y_pred)
loss3

# 실습
- 0 ~ 3으로 구성된 10개의 정답 y_true를 랜덤하게 선언하세요.  
(10, 4) 형식의 값을 np.random.randn 함수를 이용해 구하고 softmax 함수를 이용해 확률 y_pred를 구하세요.  
y_true를 one-hot으로 변경해서 (10, 4)의 확률 분포로 변경하세요.  
y_true의 확률분포와 y_pred를 이용해 cross entropy loss를 직접 구하세요.
- CategoricalCrossentropy를 이용해 cross entropy loss를 구하세요.
- SparseCategoricalCrossentropy를 이용해 cross entropy loss를 구하세요.

